In [1]:
%pip install gym gym-anytrading gym-trading-env stable-baselines3
import gymnasium as gym

from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3 import DQN

# do not remove!
from gym_trading_env.environments import TradingEnv
from gym_trading_env.renderer import Renderer

import numpy as np

from pandas.core.frame import DataFrame
from pandas._libs.tslibs.timestamps import Timestamp

import sys
sys.path.append('../../')

from coin_data import get_coin_data


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Implement gym-trading env
https://gym-trading-env.readthedocs.io/en/latest/rl_tutorial.html

In [2]:
from agents.rl_agent.util import eval_model, get_env, preprocess_data, train_model


data = get_coin_data('BTC/USDT', '1h', start_date=Timestamp('2021-01-01'))

coin_data = preprocess_data(data, window_size=60)

env = get_env(
    'BTC/USDT', 
    coin_data, 
    window_size=60, 
    positions=[-3, -2, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 2, 3],
)
model = DQN('MlpPolicy', env, learning_starts=100, verbose=1, tau=0.9, exploration_initial_eps=0.5, exploration_fraction=0.1, exploration_final_eps=0.05)
train_model('DQN', model, env, 100)

eval_model('DQN', model, env, coin_data, True)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
{'idx': 59, 'step': 0, 'date': numpy.datetime64('2021-01-03T12:00:00.000000000'), 'position_index': 4, 'position': -0.5, 'real_position': -0.5, 'data_open': 34413.53, 'data_volume': 4546.283481, 'data_low': 33928.75, 'data_close': 34103.72, 'data_high': 34600.0, 'portfolio_valuation': 10000.0, 'portfolio_distribution_asset': 0, 'portfolio_distribution_fiat': 15000.0, 'portfolio_distribution_borrowed_asset': 0.14661157199273275, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0, 'portfolio_distribution_interest_fiat': 0, 'reward': 0}
Market Return : 97.34%   |   Portfolio Return : -99.84%   |   Position Changes : 6641   |   Episode Length : 29530   |   
{'idx': 29588, 'step': 29529, 'date': numpy.datetime64('2024-05-18T11:00:00.000000000'), 'position_index': array(8), 'position': 0.5, 'real_position': 0.5002906084769163, 'data_open': 67222.21, 'data_volume':

In [4]:
eval_model('DQN', model, env, coin_data, True, True)

{'idx': 59, 'step': 0, 'date': numpy.datetime64('2021-01-03T12:00:00.000000000'), 'position_index': 8, 'position': 0.5, 'real_position': 0.5, 'data_open': 34413.53, 'data_volume': 4546.283481, 'data_low': 33928.75, 'data_close': 34103.72, 'data_high': 34600.0, 'portfolio_valuation': 10000.0, 'portfolio_distribution_asset': 0.14661157199273275, 'portfolio_distribution_fiat': 5000.0, 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0, 'portfolio_distribution_interest_fiat': 0, 'reward': 0}
Market Return : 97.34%   |   Portfolio Return : -99.80%   |   Position Changes : 6612   |   Episode Length : 29530   |   
{'idx': 29588, 'step': 29529, 'date': numpy.datetime64('2024-05-18T11:00:00.000000000'), 'position_index': array(10), 'position': 1, 'real_position': 0.9999999999999998, 'data_open': 67222.21, 'data_volume': 730.40325, 'data_low': 67186.73, 'data_close': 67301.22, 'data_high': 67400.01, 'portfolio_valuati

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/May/2024 13:17:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2024 13:17:54] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/May/2024 13:17:56] "GET /update_data/USDT_2024-05-18_13-16-57.pkl HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2024 13:17:56] "GET /metrics HTTP/1.1" 200 -


!!! Server running http://127.0.0.1:5000 !!!


In [5]:
# make reward funcition
def reward_function(history):
    return np.log(history["portfolio_valuation", -1] / history["portfolio_valuation", -2])

In [6]:
env = gym.make(
    "TradingEnv",
    name="BTC/USD",
    df=coin_data,
    windows=5,
    positions=[-5, -4, -3, -2, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 2, 3, 4, 5],
    trading_fees=0.001,
    borrow_interest_rate=0.0003/100,  # 0.003% per timestamp (one timestamp = 1 hour)
    reward_function=reward_function,
    portfolio_initial_value=1000,
)

In [7]:

env.unwrapped.add_metric('Position Changes', lambda history : np.sum(np.diff(history['position']) != 0) )
env.unwrapped.add_metric('Episode Length', lambda history : len(history['position']) )

done, truncated = False, False
observation, info = env.reset()
print(info)
while not done and not truncated:
    action = env.action_space.sample()
    observation, reward, done, truncated, info = env.step(action)

env.render()

env.unwrapped.save_for_render()

{'idx': 4, 'step': 0, 'date': numpy.datetime64('2021-01-03T15:00:00.000000000'), 'position_index': 11, 'position': 0.75, 'real_position': 0.75, 'data_high': 33873.45, 'data_open': 33811.54, 'data_volume': 8391.249757, 'data_close': 33506.62, 'data_low': 32727.0, 'portfolio_valuation': 1000.0, 'portfolio_distribution_asset': 0.02238363642766713, 'portfolio_distribution_fiat': 250.0, 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0, 'portfolio_distribution_interest_fiat': 0, 'reward': 0}


Market Return : 23.01%   |   Portfolio Return : -100.00%   |   Position Changes : 24146   |   Episode Length : 25746   |   


In [8]:
# renderer = Renderer(render_logs_dir="render_logs/BTC")
# renderer.run()

In [9]:
## Multi Dataset Trading environment
# TODO: add data from more exchanges

TODO:
- implement agent. No idea what the openAI network looks like
- consider Random Forest model
- consider XGBoost model
- stacking multiple diverse models, and a meta model on top of that
- observe copilot's demoralizing tips when it comes to trading 🤔🤔🤔 
- consider RNN
- feed the results of a time series model (ARIMA, GARCH) as input features for a machine learning model
- Markov chain model
- stop making fun of me, copilot! 🤬🤬🤬
- consider LSTM model, https://sb3-contrib.readthedocs.io/en/master/modules/ppo_recurrent.html, https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/

## Implement DQN agent

In [12]:
from stable_baselines3.common.monitor import Monitor

coin_data = preprocess(data)
env.reset()

model = DQN('MlpPolicy', env, learning_starts=100, verbose=1, tau=0.9, exploration_initial_eps=0.5, exploration_fraction=0.1, exploration_final_eps=0.05, device='cpu')

# Wrap the environment with Monitor
eval_env = Monitor(env)

eval_callback = EvalCallback(
    eval_env, best_model_save_path='./logs/',
    log_path='./logs/', eval_freq=100,
    n_eval_episodes=1,
    deterministic=True, render=False
)

model.learn(total_timesteps=1e4, callback=eval_callback)

env.render()

env.unwrapped.save_for_render()
renderer = Renderer(render_logs_dir="render_logs/BTC")
renderer.run()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Market Return : 23.01%   |   Portfolio Return : 11.45%   |   Position Changes : 1   |   Episode Length : 25746   |   
Eval num_timesteps=100, episode_reward=0.11 +/- 0.00
Episode length: 25745.00 +/- 0.00
----------------------------------
| eval/               |          |
|    mean_ep_length   | 2.57e+04 |
|    mean_reward      | 0.108    |
| rollout/            |          |
|    exploration_rate | 0.455    |
| time/               |          |
|    total_timesteps  | 100      |
----------------------------------
New best mean reward!
Market Return : 23.01%   |   Portfolio Return : 11.37%   |   Position Changes : 1   |   Episode Length : 25746   |   
Eval num_timesteps=200, episode_reward=0.11 +/- 0.00
Episode length: 25745.00 +/- 0.00
----------------------------------
| eval/               |          |
|    mean_ep_length   | 2.57e+04 |
|    mean_reward      | 0.108    |
| rollout/         

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [13]:
# load best model
model = DQN.load("logs/best_model.zip")

done, truncated = False, False
observation, info = env.reset()
print(info)
while not done and not truncated:
    action, _ = model.predict(observation)
    observation, reward, done, truncated, info = env.step(action)
    if done or truncated:
        print(info)

env.render()
env.unwrapped.save_for_render()

renderer = Renderer(render_logs_dir="render_logs/BTC")
renderer.run()

{'idx': 4, 'step': 0, 'date': numpy.datetime64('2021-01-03T15:00:00.000000000'), 'position_index': 7, 'position': -0.25, 'real_position': -0.25, 'data_high': 33873.45, 'data_open': 33811.54, 'data_volume': 8391.249757, 'data_close': 33506.62, 'data_low': 32727.0, 'portfolio_valuation': 1000.0, 'portfolio_distribution_asset': 0, 'portfolio_distribution_fiat': 1250.0, 'portfolio_distribution_borrowed_asset': 0.0074612121425557095, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0, 'portfolio_distribution_interest_fiat': 0, 'reward': 0}
Market Return : 23.01%   |   Portfolio Return : -100.00%   |   Position Changes : 17424   |   Episode Length : 25746   |   
{'idx': 25749, 'step': 25745, 'date': numpy.datetime64('2023-12-12T22:00:00.000000000'), 'position_index': array(1), 'position': -4, 'real_position': -4.055608217957823, 'data_high': 41220.38, 'data_open': 41104.02, 'data_volume': 665.07274, 'data_close': 41216.95, 'data_low': 41079.48, 'portfolio_v

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Dec/2023 00:14:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2023 00:14:46] "GET /update_data/USD_2023-12-13_00-14-40.pkl HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2023 00:14:47] "GET /metrics HTTP/1.1" 200 -
